In [1]:
# Question about Timescale we want the model to answer

import os
import sys
#parent_dir = os.path.dirname(os.path.realpath(__file__))
parent_dir = os.path.dirname(os.path.realpath(os.path.abspath('..\\')))
print(parent_dir)
# Add the parent directory to sys.path
sys.path.append(parent_dir)
from extractors.ocr_data_extractor import OCRHelper
from ocr.transformer import GPT4VisionManager

import os
import pandas as pd
import psycopg2
import numpy as np 
import re
from pgvector.psycopg2 import register_vector
from psycopg2 import sql
from psycopg2.extras import execute_batch, execute_values
import openai
import os
from openai import AzureOpenAI
import pathlib
from pathlib import Path
import os
import time
import numpy as np
import json
from psycopg2.extensions import register_adapter, AsIs
from pgvector.psycopg2 import register_vector
import camelot
from PIL import Image


C:\Users\Ashish.Madkaikar\OneDrive - Emerson\Source\ai_agents


2024-03-08 20:50:18,976 - micro - MainProcess - INFO     PDFHelper initialized. (pdf_data_extractor.py:__init__:20)


In [1]:
from extractors.ocr_data_extractor import OCRHelper
import pytesseract
import cv2
def extract_text_from_images_os_opencv(self, file_path: str) -> None:
        """
        Processes a single image file and extracts all text .
        Args:
            file_path (str): Path to the image file.
            output_path (str): Directory where the images will be saved.
        """
        with Image.open(file_path) as img:
            # Perform OCR on the image using Tesseract
            # Load image, grayscale, apply sharpening filter, Otsu's threshold 
            image = cv2.imread(file_path)
            img = cv2.medianBlur(image,5)
            ret,th1 = cv2.threshold(img,127,255,cv.THRESH_BINARY)
            th2 = cv2.adaptiveThreshold(img,255,cv.ADAPTIVE_THRESH_MEAN_C,\
                        cv2.THRESH_BINARY,11,2)
            th3 = cv2.adaptiveThreshold(img,255,cv.ADAPTIVE_THRESH_GAUSSIAN_C,\
                        cv2.THRESH_BINARY,11,2)

            gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
            sharpen_kernel = np.array([[-1,-1,-1], [-1,9,-1], [-1,-1,-1]])
            sharpen = cv2.filter2D(gray, -1, sharpen_kernel)
            thresh = cv2.threshold(sharpen, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)[1]

            ocr_text = pytesseract.image_to_string(thresh)
            return ocr_text

OCRHelper = OCRHelper()
pageno = 1
working_directory = os.getcwd() + "\\data\\"
inputfile = f"valve.png"
file = f"{working_directory}\\{inputfile}"
#text = OCRHelper.extract_text_from_images_os(file)
text = OCRHelper.extract_text_from_images_os_opencv(file)

text

ModuleNotFoundError: No module named 'extractors'

In [15]:
from lib import extract_tables_from_pdf
from ocr.transformer import GPT4VisionManager
import pathlib
import os
import time
def run_gpt4v_image_fisher(inputfile, pageno):
    gpt_vision_client = GPT4VisionManager()
    gpt_vision_client.load_environment_variables_from_env_file()
    pdf_images_path = inputfile.replace(".pdf","")
    working_directory = os.getcwd()
    image_file_path = f"{working_directory}\\tempimages\\{pdf_images_path}\\{pdf_images_path}-page-{pageno}.png"

    working_directory = os.getcwd() + "\\data\\"
    file = f"{working_directory}\\{inputfile}"
    tables = extract_tables_from_pdf(file, f"{pageno}",'stream')
    #table_question = False
    table_text = ""
    try: 
        for table in tables:
            table_text += table.df.to_string(index=False)        
    except Exception as e:
        table_text = "No table found"
    
    #table_text = parse_tables(inputfile, pageno, {})
    
    sys_message = "'You are an AI assistant capable of processing and summarizing complex documents with diagrams.'"
    user_prompt = "Please analyze this document ignoring information in tables and provide the information in the following format:\\r\\n1. Summary: Provide a concise summary of the document, focusing on the main points and overall context.\\r\\n2. Content: Your task is to extract all information from the document in a detailed and granular manner. Pay special attention to diagrams. Ensure that no information is omitted and no tables are summarized. Avoid summarizing; instead, be explicit with the details. Take your time, as the purpose is to thoroughly record all the information from the document.\\r\\n3. Category: List key categories or keywords, with a focus on main products or concepts mentioned in the document. Categories should be abstracted and listed, separated by commas, with a maximum of 10 words.\\r\\nThe purpose is to enable another system to read and understand this information in detail, to facilitate answering precise questions based on the document\'s context.\\r\\nPlease return the information in the following format:\\r\\n\\r\\n#summary\\r\\n<summary text>\\r\\n\\r\\n#content\\r\\n<content text>\\r\\n\\r\\n#category\\r\\n[<category 1>, <category 2>, <category 3>, ...]"

    extracted_context = gpt_vision_client.call_gpt4v_image(
        image_file_path,
        system_instruction=sys_message,
        user_instruction=user_prompt,
        ocr=True,
        use_vision_api=True,
        display_image=False,
        max_tokens=2000,
        seed=42,
    )
    #time.sleep(61)
    return extracted_context, table_text

In [16]:
from lib import get_openai_embeddings
#from lib import 
import os
#from embeddings import load_bert_model,get_bert_embeddings
from lib import get_metadata_from_context
model = "bert"
model_path = os.getcwd() + f"\\models_reinitialized\\{model}"
tokenizer_path = model_path + "\\tokenizer\\"
schema = os.environ["DATABASE_SCHEMA"]
table = os.environ["DATABASE_TABLE"]

pageno=1
option = "instruction-manual-fisher-et-eat-easy-e-valves-cl125-through-cl600-en-124782.pdf"

extracted_context, table_text = run_gpt4v_image_fisher(option, pageno)
#print(extracted_context, table_text)
working_directory = os.getcwd() + "\\tempimages\\"
FOLDER  = option.replace(".pdf","")
file_images_directory = f"{working_directory}\\{FOLDER}"
print(file_images_directory)
# meta,context = get_metadata_from_context(extracted_context, source=option, pageno=pageno,image_dir=file_images_directory, pdf_name=FOLDER)
# print(meta,context)
# num_tokens,cost = insert_bert_embeddings_postgresql(meta = meta,context=context,pageno=pageno, schema=schema, table = table, table_text= table_text,model=model,tokenizer=tokenizer)


2024-03-14 12:15:14,672 - micro - MainProcess - INFO     Preparing instruction for GPT-4 Vision API call. (transformer.py:prepare_instruction:119)
2024-03-14 12:15:14,674 - micro - MainProcess - INFO     Instruction: [{'role': 'system', 'content': [{'type': 'text', 'text': "'You are an AI assistant capable of processing and summarizing complex documents with diagrams.'"}]}, {'role': 'user', 'content': [{'type': 'text', 'text': "Please analyze this document ignoring information in tables and provide the information in the following format:\\r\\n1. Summary: Provide a concise summary of the document, focusing on the main points and overall context.\\r\\n2. Content: Your task is to extract all information from the document in a detailed and granular manner. Pay special attention to diagrams. Ensure that no information is omitted and no tables are summarized. Avoid summarizing; instead, be explicit with the details. Take your time, as the purpose is to thoroughly record all the information 

AttributeError: 'tuple' object has no attribute 'tb_frame'

In [32]:
from pathlib import Path 
from PIL import Image
full_path = "C:\\Users\\Ashish.Madkaikar\\OneDrive - Emerson\\Source\\ai_agents\\examples\\OS_ChatWithYourDocumentationAgent\\tempimages\\\\instruction-manual-fisher-et-eat-easy-e-valves-cl125-through-cl600-en-124782\\instruction-manual-fisher-et-eat-easy-e-valves-cl125-through-cl600-en-124782-page-29.png"
def get_relative_path(full_path):
    path = Path(full_path)
    
    return path.relative_to(path.parent.parent.parent)
pa = get_relative_path(full_path)


print(str(pa))
#img = Image.open(pa)
#img.show()

tempimages\instruction-manual-fisher-et-eat-easy-e-valves-cl125-through-cl600-en-124782\instruction-manual-fisher-et-eat-easy-e-valves-cl125-through-cl600-en-124782-page-29.png


In [ ]:
meta,context = get_metadata_from_context(extracted_context, source=option, pageno=pageno,image_dir=file_images_directory, pdf_name=FOLDER)

In [15]:
import openai


In [124]:
import openai
import os
from openai import AzureOpenAI
openai.api_type = "azure"
openai.api_version = os.environ["AZURE_OPENAI_API_VERSION"] 
openai.api_version = os.environ["AZURE_OPENAI_API_VERSION"] 
openai.api_base = os.environ["AZURE_OPENAI_ENDPOINT_VISION"]
openai.api_key = os.environ["AZURE_OPENAI_API_KEY"]

def summarize_fisher(extracted_context, model="gpt-4"):
    '''
    returns response to the question asked 

    extracted_context:
    table_text:str , text extrac
    questions:list or array of str,  questions to be asked
    q:int , question index for asking specific question at the given index  

    '''
    sys_message = "'You are an AI assistant capable of processing and summarizing complex documents with diagrams.'"
    user_prompt = f"Please analyze give text - {extracted_context} ignoring information in tables and provide the information in the following format:\\r\\n1. Summary: Provide a concise summary of the document, focusing on the main points and overall context.\\r\\n2. Content: Your task is to extract all information from the document in a detailed and granular manner. Pay special attention to diagrams. Ensure that no information is omitted and no tables are summarized. Avoid summarizing; instead, be explicit with the details. Take your time, as the purpose is to thoroughly record all the information from the document.\\r\\n3. Category: List key categories or keywords, with a focus on main products or concepts mentioned in the document. Categories should be abstracted and listed, separated by commas, with a maximum of 10 words.\\r\\nThe purpose is to enable another system to read and understand this information in detail, to facilitate answering precise questions based on the document\'s context.\\r\\nPlease return the information in the following format:\\r\\n\\r\\n#summary\\r\\n<summary text>\\r\\n\\r\\n#content\\r\\n<content text>\\r\\n\\r\\n#category\\r\\n[<category 1>, <category 2>, <category 3>, ...]"
     
    response = openai.chat.completions.create(
                            model=model,
                            messages=[
                                {"role": "system", "content": sys_message},
                                {"role": "user", "content": user_prompt}
                            ],
                            temperature=0.7,
                            max_tokens=1000,
                        )
    #st.write(response.choices)
    #answer = response.choices[0].message.content
    return response.choices[0].message.content

def is_table_normalized(table_text, model="gpt-4"):
    #print(table_text)
    '''
    returns normalized table response to the question asked 

    extracted_context:
    table_text:str , text extrac
    questions:list or array of str,  questions to be asked
    q:int , question index for asking specific question at the given index  

    '''

    system_message = "You are a specialized AI programs that normalizes complex table structures while maintaining the relationships between columns and rows in the table."
    
    prompt = f"""Analyze the {table_text} and return whether the table is normalized like a matrix and what is your confidence score, lett the answer be very short and brief"""

    response = openai.chat.completions.create(
                            model=model,
                            messages=[
                                {"role": "system", "content": system_message},
                                {"role": "user", "content": prompt}
                            ],
                            temperature=0.7,
                            max_tokens=1000,
                        )
    #st.write(response.choices)
    #answer = response.choices[0].message.content
    return response.choices[0].message.content


def normalize_table(table_text, model="gpt-4"):
    #print(table_text)
    '''
    returns normalized table response to the question asked 

    extracted_context:
    table_text:str , text extrac
    questions:list or array of str,  questions to be asked
    q:int , question index for asking specific question at the given index  

    '''

    system_message = "You are a specialized AI programs that normalizes complex table structures while maintaining the relationships between columns and rows in the table."
    
    prompt = f"""Normalize the given table : {table_text} and return the normalized table with consistent structure where each row represents a record and each column represents an attribute of that record without any explaination or additional information"""

    # prompt = f"""Using context: {extracted_context}, find the answer for the following question: {questions[q]} then again verify your answer for question:{questions[q]} using this additional context:{table_text}, after comparing your two answers, without mentionining and explaining how you compared or verified give precise answer without repeatation of information in your answer. 
    # Please limit your response to the information provided in the context. If there isn't enough information, state that more is needed. To personalize your response, include emoticons."""

    response = openai.chat.completions.create(
                            model=model,
                            messages=[
                                {"role": "system", "content": system_message},
                                {"role": "user", "content": prompt}
                            ],
                            temperature=0.7,
                            max_tokens=1000,
                        )
    #st.write(response.choices)
    #answer = response.choices[0].message.content
    return response.choices[0].message.content


In [6]:
import pdfplumber
from pdfplumber.utils.pdfinternals import resolve_and_decode, resolve
from lib import respond_fisher,run_gpt4v_image_fisher
import camelot
import os
import pandas as pd

#extracted_context, table_text = run_gpt4v_image_fisher(option, pageno)
#normalize_table = normalize_table(table)
# question = "What is the gasket kit part number of a 8 inch ET control valve with whisper Trim I?"
# answer = respond_fisher(extracted_context=extracted_context, table_text=table, questions=[question], q=0,model="gpt-4" )
# answer
# text = page.extract_text(keep_blank_chars=True)
# print(text)
# flavor = "lattice"
# text = ""
# #lines = page.extract_text_lines(layout=False, strip=True, return_chars=True)
# display(table)
# # for row in table:
#     print(row)
#pageno=1
option = "instruction-manual-fisher-et-eat-easy-e-valves-cl125-through-cl600-en-124782.pdf"



def extract_tables_pdfplumber(filepath, pageno, table_settings, show_debug, normalize_tables):
    tabletext = ""
    if table_settings == None:
        table_settings = {
            "vertical_strategy": "lines", 
            "horizontal_strategy": "lines",
            "explicit_vertical_lines": [],
            "explicit_horizontal_lines": []
        }

    try:
        pdf = pdfplumber.open(filepath)
        page = pdf.pages[pageno]
        im = page.to_image()
        if show_debug:
            im.reset().debug_tablefinder(table_settings)
            display(im.debug_tablefinder())
        tabletext = page.extract_table(table_settings)
    except Exception as e :
        print(e)
        tabletext = "Error while parsing PDF, either file is not in right format or not present or pageno doesnot exist"  
    finally:
        pdf = ""
          

    return tabletext

pageno=25
working_directory = os.getcwd() + "\\data\\"
#inputfile = "p2_fds_e431.pdf"
#inputfile = f"d103428x012.pdf"
#inputfile = f"fisher_ET__26_Gasket_modified_Table.pdf"
inputfile = f"instruction-manual-fisher-et-eat-easy-e-valves-cl125-through-cl600-en-124782.pdf"
file = f"{working_directory}\\{inputfile}"

#display(im.debug_tablefinder())
table_settings = {
    "vertical_strategy": "lines", 
    "horizontal_strategy": "lines",
    "explicit_vertical_lines": [],
    "explicit_horizontal_lines": []
}
table = extract_tables_pdfplumber(file,pageno, table_settings=table_settings, show_debug=False)
display(table)




[['DESCRIPTION',
  'Standard Trim Cage\nWhisper Trim I Cage\nCavitrol III - 1 Stage Cage',
  'Cavitrol III - 2 Stage Cage\nWhisper Trim III Cage\nWhisperFlo Cage'],
 [None,
  '-198 to 593(cid:2)C (-325 to 1100(cid:2)F)',
  '-198 to 593(cid:2)C (-325 to 1100(cid:2)F)'],
 ['Full Capacity Valves', 'Part Number', 'Part Number'],
 ['NPS 1 and 1‐1/4\nNPS 1‐1/2 (NPS 2 EAT)\nNPS 2\nNPS 2‐1/2 (NPS 3 EAT)\nNPS 3 (NPS 4 EAT)\nNPS 4 (NPS 6 EAT)\nNPS 6\nNPS 8',
  'RGASKETX162\nRGASKETX172\nRGASKETX182\nRGASKETX192\nRGASKETX202\nRGASKETX212\nRGASKETX222\nRGASKETX232',
  'RGASKETX422\nRGASKETX432\nRGASKETX442\nRGASKETX452\nRGASKETX462\nRGASKETX472\nRGASKETX482\n10A3265X152'],
 ['Restricted Capacity Valves w/ Metal Seating', '', ''],
 ['NPS 1‐1/2 x 1 (NPS 2 x 1 EAT)\nNPS 2 x 1\nNPS 2‐1/2 x 1‐1/2 (NPS 3 x 1‐1/2 EAT)\nNPS 3 x 2 (NPS 4 x 2 EAT)\nNPS 4 x 2‐1/2 (NPS 6 x 2‐1/2 EAT)',
  'RGASKETX242\nRGASKETX252\nRGASKETX262\nRGASKETX272\nRGASKETX282',
  '- - -\n- - -\n- - -\n- - -\n- - -']]

In [1]:
normalized_table = normalize_table(table)
print(normalized_table)
question = "What is the gasket kit part number of a 8 inch ET control valve with whisper Trim I?"
answer = respond_fisher(extracted_context=extracted_context, table_text=normalized_table, questions=[question], q=0,model="gpt-4" )
answer


NameError: name 'normalize_table' is not defined

In [139]:


tb = is_table_normalized(table)
print(tb)

The table is not normalized like a matrix. Confidence score: High.

Normalization issues include:
- The first row contains multiple values in a single cell.
- The rows represent different sizes but share a structure that is not consistent for the last row (the replacement value '10A3265X152' does not follow the 'RGASKETX4xx' pattern).
- The table combines different types of data without separate fields (sizes, part numbers, and temperature ranges).


In [132]:
table

[['Standard Designation', 'Common Name or Tradename'],
 ['CoCr‐A Hardfacing Alloy\nR30006\nS17400 SST\nS31600 SST',
  'CoCr‐A\nAlloy 6 Casting\n17‐4PH Stainless Steel\n316 Stainless Steel'],
 ['S41000 SST\nS41600 SST\nWCC Carbon Steel Casting',
  '410 Stainless Steel\n416 Stainless Steel\nWCC']]

In [80]:
from pdfplumber.structure import PDFStructTree
pageno = 25
page = pdf.pages[pageno]
stree = PDFStructTree(pdf, page)
img = page.to_image()
img.draw_rects(stree.element_bbox(td) for td in table.find_all("TD"))

StructTreeMissing: PDF has no structure

In [7]:
display(extracted_context) 
display(table_text)
display(table)

'#summary\nThe document is an instruction manual for Fisher™ ET and EAT easy-e™ Valves, with a focus on models CL125 through CL600. It provides guidelines for installation, maintenance, and parts information specifically for NPS 1 through 8 Fisher ET valves, and NPS 1 through 6 EAT valves with CL600 ratings. The manual emphasizes the necessity of being fully trained and qualified before installing, operating, or maintaining these valves to avoid personal injury or property damage. It includes a list of contents and a cautionary notice, along with a figure depicting the Fisher ET Control Valve with a 667 Actuator.\n\n#content\nThe document outlines the following sections in its contents:\n- Introduction\n- Scope of Manual\n- Description\n- Specifications\n- Educational Services\n- Installation\n- Maintenance\n- Packing Lubrication\n- Packing Maintenance\n- Replacing Packing\n- Trim Maintenance\n- Disassembly\n- Lapping Metal Seats\n- Valve Plug Maintenance\n- Assembly\n- ENVIRO-SEAL™ Be

'                                                                                    0  1\n                                                                  CL125 through CL600   \n                                                                             Contents   \n  Introduction\\n. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .  1\n       Scope of Manual\\n. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .  1\n   Description\\n. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .  3\n    Specifications\\n. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .  3\n          Educational Services\\n. . . . . . . . . . . . . . . . . . . . . . . . . . .  3\nInstallation\\n. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .  4\n   Maintenance\\n. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .  5\n         Packing Lubrication\\n. . . . . . . . . . . . . . . . . . . . . . . . . . . .  6\n 

[['VALVE SIZE, NPS', 'STAGE', 'WEIGHT', None],
 [None, None, 'kg', 'lb'],
 [None, '2-stage', '63', None],
 [None, '3-stage', '68', None],
 [None, '2-stage', '93', None],
 [None, '3-stage', '105', None],
 [None, '2-stage', '212', None],
 [None, '3-stage', '234', None],
 [None, '3-stage', '518', None]]

('The available shut off classifications for ET valves with Cavitrol III 1‐stage cage Metal are IV (standard) and V (optional). 😊',
 'What are the availabe shut off classifications for ET valves with Cavitrol III 1‐stage cage Metal?')